# Calculate Urban Metrics: Fullness
Fraction of the total area of the urban extent that is built-up. Use a custom script that takes the mean value of all the pixels of the GHS built-up within the urban extent. The GHS built-up layer values are expressed as decimals from 0 to 1, and correspond to the fraction of the pixel that is covered by a building (Pesaresi et al., 2016)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import time

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [ ]:
from shapely.geometry import mapping
from shapely.geometry import Point

In [ ]:
# Get reference to GOSTNets
sys.path.append(r"C:\repos\INFRA_SAP")
from infrasap.urban_metrics import *

In [ ]:
import rasterio
from rasterio.mask import mask
from rasterio import Affine  # or from affine import Affine

In [ ]:
start_time = time.time()

## Inputs are GHS built and the urban extents
Mollweide projection should work good because it is an equal-area projection

In [ ]:
built_up_layer = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\GHS_BUILT_LDS2014_GLOBE_R2018A_54009_250_V2_0\GHS_BUILT_LDS2014_GLOBE_R2018A_54009_250_V2_0.tif"
# built_up_layer = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\2015_250m_GHS_Built\UZB_2015_GHS_built_merged.tif"

In [ ]:
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\russia_urban_extents_merged_mollweide.shp"
# shpName = r"C:\repos\GOST_Urban\Notebooks\Implementations\eca_wo_rus_urban_clusters_ghs_pop_smooth_100k_mollweide2.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_FUAs_Project_Mollweide.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_ghs_built_up_extents_4326\UZB_only_ghs_built_up_extents_mollweide_geom_fixed_greater_50k.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_ghs_built_up_extents_4326\UZB_ghs_built_up_extents_mollweide_geom_fixed.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\ECA_wo_rus_urban_extents\eca_wo_rus_built_up_extents_molleweide.shp"
shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\all_urban_clusters_5k_up_molleweide.shp"

In [ ]:
input_shapes_gpd = gpd.read_file(shpName)
input_shapes_gpd.head(5)

In [ ]:
%%time

with rasterio.open(built_up_layer) as src:
    pixelSizeX, pixelSizeY = src.res
    # print(pixelSizeX, pixelSizeY)
    pixel_area = 250**2

    input_shapes_gpd = gpd.read_file(shpName)

    # pseudocode
    # For each Shape:
    # Select all built-up pixels that are mostly within shape
    # Area of shape = sum of all pixels * area of each pixel
    # Built-up Area = (sum pixel value of each pixel) / 100
    # Fullness index = Built-up Area / Area of Shape

    # for entry in input_shapes_gpd.head(5).iterrows():
    for entry in input_shapes_gpd.iterrows():
        print(entry[0])
        # print(row[1]['geometry'])

        # extract the geometry in GeoJSON format
        geometry = entry[1]["geometry"]  # list of shapely geometries
        # geometry = geoms[0] # shapely geometry
        geoms = [mapping(geometry)]

        # extract the raster values values within the polygon
        out_image, out_transform = mask(src, geoms, crop=True, nodata=-9999.0)
        data = out_image[0, :, :]

        row, col = np.where(data != -9999.0)
        val = np.extract(data != -9999.0, data)

        T1 = out_transform * Affine.translation(0.5, 0.5)  # reference the pixel centre
        rc2xy = lambda r, c: (c, r) * T1

        d = gpd.GeoDataFrame({"col": col, "row": row, "val": val})

        # coordinate transformation
        d["x"] = d.apply(lambda row: rc2xy(row.row, row.col)[0], axis=1)
        d["y"] = d.apply(lambda row: rc2xy(row.row, row.col)[1], axis=1)

        # geometry
        d["geometry"] = d.apply(lambda row: Point(row["x"], row["y"]), axis=1)

        # Area of shape = sum of all pixels * area of each pixel
        area_of_shape = pixel_area * d.count()[0]
        # Built-up Area = (sum pixel value of each pixel) / 100
        d2 = d.val / 100
        built_up = sum(d2) * pixel_area
        # Fullness index = Built-up Area / Area of Shape
        fullness_index = built_up / area_of_shape
        print(f"fullness index: {fullness_index}")

        # creates a temporary GDF for just the row's shape
        temp_gdf = input_shapes_gpd.iloc[[entry[0]]]

        # print("print temp_gdf")
        # print(temp_gdf)

        # Put all metrics in a DataFrame
        metrics_scalar = {}
        metrics_scalar["fullness_index"] = [fullness_index]
        metrics_df = pd.DataFrame(metrics_scalar)

        # print("print metrics_scalar")
        # print(metrics_scalar)

        # and concatenate it with the row's shape
        new_temp_gdf = pd.concat([temp_gdf.reset_index(drop=True), metrics_df], axis=1)

        # print("print new_temp_gdf")
        # print(new_temp_gdf)
        # print(entry[0])
        # put the results of each row into a new DataFrame
        if entry[0] == 0:
            print("new_temp_gdf")
            output_new_temp_gdf = new_temp_gdf
        else:
            output_new_temp_gdf = output_new_temp_gdf.append(
                new_temp_gdf, ignore_index=True
            )

In [ ]:
output_new_temp_gdf.crs

In [ ]:
# make the GeoDataFrame unprojected
output_new_temp_gdf = output_new_temp_gdf.to_crs("epsg:4326")

In [ ]:
output_new_temp_gdf

In [ ]:
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_metrics_results_russia"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_urban_metrics_results_wo_rus"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_GHS_FUAs_results"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_GHS_urban_extents_results"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_GHS_urban_extents_results_all"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_urban_metrics_results_wo_rus_all"
output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\all_urban_extents_results_5k_up"

In [ ]:
# save as CSV
# output_new_temp_gdf.to_csv(output + r"\ECA_russia_urban_metrics_100k_fullness.csv")
# output_new_temp_gdf.to_csv(output + r"\UZB_only_urban_metrics_FUAs_fullness.csv")
# output_new_temp_gdf.to_csv(output + r"\UZB_only_urban_metrics_urban_extents_fullness.csv")
# output_new_temp_gdf.to_csv(output + r"\UZB_only_urban_metrics_urban_extents_all_fullness.csv")
# output_new_temp_gdf.to_csv(output + r"\ECA_wo_rus_urban_metrics_urban_extents_all_fullness.csv")
output_new_temp_gdf.to_csv(output + r"\all_urban_metrics_5k_up_fullness.csv")

In [ ]:
print(f"total time to process: {time.time()-start_time}")